# Import libraries

In [0]:
import time
t = time.time()
import string
import random
import json
import math
import random
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelBinarizer
import tensorflow as tf
import matplotlib
from matplotlib import pyplot as plt
%matplotlib inline

In [2]:
# pip intall gensim library
!pip install --upgrade gensim
from gensim.models import Word2Vec

    100% |████████████████████████████████| 22.6MB 60kB/s 
Requirement already up-to-date: numpy>=1.11.3 in /usr/local/lib/python3.6/dist-packages (from gensim)
    100% |████████████████████████████████| 50.0MB 27kB/s 
  Using cached smart_open-1.5.7.tar.gz
Requirement already up-to-date: six>=1.5.0 in /usr/local/lib/python3.6/dist-packages (from gensim)
  Using cached boto3-1.6.21-py2.py3-none-any.whl
  Using cached boto-2.48.0-py2.py3-none-any.whl
  Using cached bz2file-0.98.tar.gz
Requirement already up-to-date: requests in /usr/local/lib/python3.6/dist-packages (from smart-open>=1.2.1->gensim)
    100% |████████████████████████████████| 61kB 9.1MB/s 
    100% |████████████████████████████████| 4.1MB 311kB/s 
Requirement already up-to-date: urllib3<1.23,>=1.21.1 in /usr/local/lib/python3.6/dist-packages (from requests->smart-open>=1.2.1->gensim)
Requirement already up-to-date: certifi>=2017.4.17 in /usr/local/lib/python3.6/dist-packages (from requests->smart-open>=1.2.1->gensim)
Re

# Load and Inspect Cleaned Data

**Authentication via google auth**

In [0]:
# Code to read csv file into colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

**Upload a file from google cloud where file ids are stored as a json**

In [4]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
  
file_id_data= json.loads(uploaded['file_id.json'])

Saving file_id.json to file_id.json
User uploaded file "file_id.json" with length 489 bytes


**Download files to the kernel**

In [0]:
downloaded1 = drive.CreateFile({'id':file_id_data["modified_train_file_id"]}) 
downloaded1.GetContentFile('train.csv')  
downloaded2 = drive.CreateFile({'id':file_id_data["modified_test_file_id"]}) 
downloaded2.GetContentFile('test.csv') 

**Load train and test data to notebook**

In [0]:
train_data = pd.read_csv('train.csv') 
test_data = pd.read_csv('test.csv')

**Inspect train and test data**

In [7]:
train_data.head(20)

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,length
0,0000997932d777bf,explan edit made usernam hardcor metallica fan...,0,0,0,0,0,0,23
1,000103f0d9cfb60f,aww match background colour seem stuck thank t...,0,0,0,0,0,0,10
2,000113f07ec002fd,hey man realli tri edit war guy constant remov...,0,0,0,0,0,0,21
3,0001b41b1c6bb37e,make real suggest improv wonder section statis...,0,0,0,0,0,0,52
4,0001d958c54c6e35,sir hero chanc rememb page,0,0,0,0,0,0,5
5,00025465d4725e87,congratul well use tool well talk,0,0,0,0,0,0,6
6,0002bcb3da6cb337,cocksuck piss around work,1,1,1,0,1,0,4
7,00031b1e95af7921,vandal matt shirvington articl revert plea ban,0,0,0,0,0,0,7
8,00037261f536c51d,sorri word nonsen offen anyway intend write an...,0,0,0,0,0,0,38
9,00040093b2687caa,align subject contrari dulithgow,0,0,0,0,0,0,4


In [8]:
print("Shape of train data:", train_data.shape)

Shape of train data: (159571, 9)


In [9]:
test_data.head(20)

,id,comment_text,length
0,00001cee341fdb12,yo bitch ja rule succ ever hate sad mofucka bi...,43
1,0000247867823ef7,rfc titl fine imo,4
2,00013b17ad220c46,sourc zaw ashton lapland,4
3,00017563c3f7919a,look back sourc inform updat correct form gues...,15
4,00017695ad8997eb,anonym edit articl,3
5,0001ea8717f6de06,thank understand think high would revert witho...,8
6,00024115d4cbde0f,plea add nonsen wikipedia edit consid vandal q...,17
7,000247e83dcc1211,dear god site horribl,4
8,00025358d4737918,fool believ number correct number lie ponder n...,44
9,00026d1092fe71cc,doubl redirect fix doubl redirect blank outer ...,21


In [10]:
print("Shape of test data:", test_data.shape)

Shape of test data: (153164, 3)


# Create Word2Vec Model

**Aggregrate all the comments from train and test data in a list**

In [11]:
train_comment = train_data["comment_text"].as_matrix()
test_comment = test_data["comment_text"].as_matrix()
all_comments = np.concatenate((train_comment, test_comment), axis = 0)
print("Shape of all comments:",all_comments.shape)
del(train_comment, test_comment)
all_comments = all_comments.tolist()

Shape of all comments: (312735,)


**Break each comment into list of words**

In [0]:
sentences = []
for comment in all_comments:
  comment = str(comment)
  sentences.append(comment.split())
del(all_comments)

**Create Word2Vec (Skip-Gram) model using gensim**

In [13]:
t3 = time.time()
size = 100
window = 25
model = Word2Vec(sentences, sg = 1, size=size, window=window, min_count=1, workers=4)
print("Finished creating word2vec model.", "Time needed: ", time.time()-t3)

Finished creating word2vec model. Time needed:  1044.8663897514343


**Save the model to the kernel**

In [0]:
fname = "word2vec_model"
model.wv.save_word2vec_format(fname)

**Save the model paramters in a json**

In [0]:
import json
data = {"size": size, "window":window}
with open('data.json', 'w') as outfile:
    json.dump(data, outfile)

**Save the Word2Vec model in google drive directory**

In [0]:
# Create & upload a text file.
uploaded = drive.CreateFile({'title': 'word2vec_model', 'parents':[{u'id': file_id_data["download_folder2_id"]}]})
uploaded.SetContentFile("word2vec_model")
uploaded.Upload()

**Save the json file in google drive directory**

In [0]:
# Create & upload a text file.
uploaded = drive.CreateFile({'title': 'data.json', 'parents':[{u'id': file_id_data["download_folder2_id"]}]})
uploaded.SetContentFile("data.json")
uploaded.Upload()